In [1]:
!pip install langchain langchain-openai openai wikipedia -q
import os
import wikipedia
from langchain_openai import ChatOpenAI

# Configurar el idioma de Wikipedia
wikipedia.set_lang("es")

# Configuración del LLM
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        openai_api_base=os.environ.get("GITHUB_BASE_URL"),
        openai_api_key=os.environ.get("GITHUB_TOKEN"),
        temperature=0
    )
    print("✅ LLM de LangChain configurado.")
except Exception as e:
    print(f"❌ Error configurando el LLM: {e}")
    llm = None

c:\Users\info\Documents\GitHub\Ingenier-a-de-Soluciones-con-Inteligencia-Artificial\venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


✅ LLM de LangChain configurado.


In [2]:
from langchain.agents import tool, create_openai_tools_agent, AgentExecutor
from langchain import hub

@tool
def saludar_usuario(query: str) -> str:
    """Saludar amablemente al usuairo por su nombre."""
    
    try:
        return f"¡Hola {query}! ¿Cómo estás hoy?"
    except Exception as e:
        return f"Ocurrió un error: {e}"

tools = [saludar_usuario]

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("✅ Agente y herramientas listos.")

ImportError: cannot import name 'tool' from 'langchain.agents' (c:\Users\info\Documents\GitHub\Ingenier-a-de-Soluciones-con-Inteligencia-Artificial\venv\Lib\site-packages\langchain\agents\__init__.py)

In [ ]:
from langchain.memory import ConversationSummaryMemory

# Esta memoria necesita un LLM para hacer los resúmenes
memory_summary = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

def chat_with_agent_summary(query: str):
    history = memory_summary.load_memory_variables({})["chat_history"]
    response = agent_executor.invoke({"input": query, "chat_history": history})
    memory_summary.save_context({"input": query}, {"output": response["output"]})
    return response["output"]

In [ ]:
print("Pregunta 1: ¿cuanto es 147 + 443?")
chat_with_agent_summary("cuanto es 147 + 443??")

print("Pregunta 2:  ¿cuanto es 915-458?")
chat_with_agent_summary("¿cuanto es 915-458?")

print("Pregunta 3: ¿Sobre qué estábamos hablando?")
response3_summary = chat_with_agent_summary("¿Sobre qué estábamos hablando?")
print(f"Respuesta 3: {response3_summary}")

In [ ]:
@tool
def saludar(nombre: str) -> str:
    """Devuelve un saludo personalizado."""
    return f"¡Hola {nombre}! Encantado de conversar contigo."

tools = [saludar]


prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)


memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

def chat_with_agent(query: str):
    history = memory.load_memory_variables({})["chat_history"]
    response = agent_executor.invoke({
        "input": query,
        "chat_history": history
    })
    memory.save_context({"input": query}, {"output": response["output"]})
    return response["output"]


print("\n Chatbot iniciado (escribe 'salir' para terminar)\n")
while True:
    user_input = input("Tú: ")
    if user_input.lower() in ["salir", "exit", "quit"]:
        print("¡Hasta luego!")
        break
    respuesta = chat_with_agent(user_input)
    print(f"Bot: {respuesta}\n")